In [5]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text") 

from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

import bs4
from langchain_community.document_loaders import WebBaseLoader

bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
bs4_strainer = bs4.SoupStrainer(id="mw-content-text")
loader = WebBaseLoader(
    web_paths=("https://vi.wikipedia.org/wiki/H%C3%A0_N%E1%BB%99i",),
    bs_kwargs={"parse_only": bs4_strainer}
)

docs = loader.load()
assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}") # Load được bao nhiêu kí tự?
#print(docs[0].page_content[:500])
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs) # chia được bao nhiêu document
print(f"Split blog post into {len(all_splits)} sub-documents.")

document_ids = vector_store.add_documents(documents=all_splits)
#print(document_ids[:3])

# Retrieval (Truy xuất) - Test similarity search

results = vector_store.similarity_search("hà nội có phải thủ đô không", k=2)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

# Tạo retriever từ vector store
retriever = vector_store.as_retriever(k=5)

# Tạo RAG chain:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# Tạo prompt template
prompt = ChatPromptTemplate.from_template(
    """Trả lời câu hỏi dựa trên context sau:
Context: {context} Câu hỏi: {question} Trả lời:""")

# Tạo model
model = ChatOllama(model="gpt-oss")

# Tạo chain
rag_chain = prompt | model

# Sử dụng
question = "Hãy tóm tắt nội dung chính"
docs = retriever.invoke(question)
context = "\n\n".join([doc.page_content for doc in docs])

response = rag_chain.invoke({"context": context, "question": question})
print(response.content)


 ········


Total characters: 160573
Split blog post into 248 sub-documents.
* Đối với các định nghĩa khác, xem Hà Nội (định hướng).



Hà Nội

 Thành phố trực thuộc trung ương
Biểu trưngQuang cảnh thành phố nhìn từ cầu Nhật TânLăng BácTháp Rùa trên hồ GươmKhuê Văn CácCột cờ Hà NộiChùa Một CộtKhu phố cổ Hà NộiNhà hát LớnBiệt danhHiện nay:
Hà Thành
Thành phố vì hòa bình[1]
Thủ đô ngàn năm văn hiến
Đất kinh kỳ
Thời Pháp thuộc: [{'source': 'https://vi.wikipedia.org/wiki/H%C3%A0_N%E1%BB%99i', 'start_index': 0}]
* Hà Nội cũng là một thành phố đặc biệt nhiều đầm hồ, là dấu vết còn lại của các dòng sông cổ. Trong khu vực nội thành, hồ Tây có diện tích lớn nhất, khoảng 500 ha, đóng vai trò quan trọng trong khung cảnh đô thị, ngày nay được bao quanh bởi nhiều khách sạn, biệt thự. Hồ Gươm nằm ở trung tâm lịch sử của thành phố, khu vực sầm uất nhất, luôn giữ một vị trí đặc biệt đối với Hà Nội. Trong khu vực nội thành có thể kể tới những hồ khác như Trúc Bạch, Thiền Quang, Thủ Lệ... Ngoài ra, còn nhiều đầm hồ